In [1]:
import pandas as pd
import requests
import warnings
warnings.filterwarnings("ignore")
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

In [12]:
def wuzzuf_job(job_name):
    # scarping jobs from WUZZUF 
    titles       = []
    companies    = []
    locations    = []
    cities       = []
    countries    = []
    times        = []
    level        = []
    experiences  = []
    skills       = []
    descriptions = []
    Requirements = []
    images       = []
    page = 0
    #while True:
    for page in range (0 , 1):
        #the link for job search with  replacing page number with page value in range 
        data  = requests.get(f'https://wuzzuf.net/search/jobs/?a=hpb%7Cspbg&q={job_name}&start={page}').text
        #page += 1
        soup  = BeautifulSoup(data)
        main  = soup.findAll("div",attrs={'class':'css-1gatmva e1v1l3u10'})  # getting all jobs in main 
        if len(main) == 0:
            break
        for a in main:
            # scraping the title 
            t     = a.find('h2') 
            title = t.text.split('-')[0].split('(')[0].strip()  # some jobs have location in the title 
            titles.append(title)
            # scraping company name
            c       = a.find('a',attrs={'class':'css-17s97q8'})
            company = c.text[:-1].strip()  # removing "-" from the name 
            companies.append(company)
            # scraping the location
            l        = a.find('span',attrs={'class':'css-5wys0k'})
            larray   = l.text.split(',')[:-1]  # splitting the location for location[0:-2], city[-2] and country[-1]
            if(len(larray)>1):
                cities.append(larray[-1])
                location = ','.join(larray[0:-1])
                locations.append(location)
            else:
                cities.append(larray[-1])
                locations.append(None)
            country  = l.text.split(',')[-1].strip()
            countries.append(country)
            # scraping type of the job
            description = a.find('div',attrs={'class':'css-y4udm8'})
            time        = description.findAll('a',attrs={'class':'css-n2jc4m'})
            t           = []        
            for T in time:
                t.append(T.find('span').text.split('/')[0].strip())
            times.append(t)
            # scraping experience, level and skills of the job
            info        = description.findAll('div')[1]
            experience  = info.find('span').text
            experiences.append(experience.replace('·',' ').strip())
            jobskills   = []
            for j , i in enumerate(info.findAll('a')):
                if(j==0):
                    level.append(i.text.strip())
                else:
                    jobskills.append(i.text.replace('·',' ').strip())
            skills.append(jobskills)
            # Description & requirements and images with selenium
            options = webdriver.ChromeOptions()
            options.headless = True
            driver = webdriver.Chrome(chrome_options = options)
            job_href = a.find('h2').find('a')['href']
            job_link = f'https://wuzzuf.net{job_href}'
            driver.get(job_link)
            description = driver.find_element(By.CLASS_NAME, "css-1uobp1k")
            descriptions.append(description.text)
            Requirement = driver.find_element(By.CLASS_NAME, "css-1t5f0fr")
            Requirements.append(Requirement.text)
            img = driver.find_element(By.CLASS_NAME, "css-jb4x6y")
            images.append(img.get_attribute('src'))
            driver.close()
    df = pd.DataFrame({'title':titles,'company name':companies,'location':locations,
                        'city':cities,'country':countries,'type':times,'level':level,
                        'experience':experiences,'skills':skills,'Description':descriptions,
                       'Requirements':Requirements,'Image link':images})
    return df

In [13]:
machine_learning_wuzzuf = wuzzuf_job('machine learning')
machine_learning_wuzzuf

,title,company name,location,city,country,type,level,experience,skills,Description,Requirements,Image link
0,AI/ML,SURE International Technology,None,Cairo,Egypt,[Full Time],Manager,5+ Yrs of Exp,"[IT/Software Development, Operations/Managemen...","Lead, mentor, and advise the ML team members.\...","Bachelor’s degree in Statistics, Computer Scie...",https://images.wuzzuf-data.net/files/company_l...
1,Senior Machine learning Engineer,Botit,Abdeen,Cairo,Egypt,[Full Time],Experienced,3 - 7 Yrs of Exp,"[IT/Software Development, Engineering - Teleco...","As a part of the AI team, the candidate will p...",Experience in using version control systems (e...,https://images.wuzzuf-data.net/files/company_l...
2,Senior Computer Vision Engineer,wobot.ai,None,New Delhi,India,"[Full Time, Work From Home]",Senior Management,3 - 6 Yrs of Exp,"[IT/Software Development, Engineering - Teleco...",Description:\nThis is a senior role where you ...,Requirements:\nProficient with Training of Det...,https://images.wuzzuf-data.net/files/company_l...
3,Senior/Mid Senior Deep Learning Engineer,Confidential,Mokattam,Cairo,Egypt,[Full Time],Experienced,4+ Yrs of Exp,"[IT/Software Development, Engineering - Teleco...","Reviewing, replicating, and mapping research p...",Completed Master’s degree or higher in Artific...,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA..."
4,Data Scientist,Confidential,None,Riyadh,Saudi Arabia,[Full Time],Experienced,2 - 4 Yrs of Exp,"[IT/Software Development, Analyst/Research, En...","Selecting features, building, and optimizing m...",Strong applied mathematical and statistical sk...,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA..."
5,Senior Data Scientist,BBI-Consultancy,Nasr City,Cairo,Egypt,[Full Time],Experienced,3 - 5 Yrs of Exp,"[IT/Software Development, Analyst/Research, En...",Identify valuable data sources and automate co...,"BSc/BA in Computer Science, Engineering or rel...",https://images.wuzzuf-data.net/files/company_l...
6,Senior Computer Vision Engineer,Atomica,None,Cairo,Egypt,"[Full Time, Work From Home]",Experienced,4+ Yrs of Exp,"[IT/Software Development, Engineering - Teleco...",Atomica.ai is a US-based company that helps do...,Bachelor's degree in the areas of Computer Sci...,https://images.wuzzuf-data.net/files/company_l...
7,Data Scientist,Seoudi Supermarket,Sheikh Zayed,Giza,Egypt,[Full Time],Entry Level,1 - 4 Yrs of Exp,"[IT/Software Development, Analyst/Research, En...","Analyze raw data: assessing quality, cleansing...",Bachelor's degree or equivalent experience in ...,https://images.wuzzuf-data.net/files/company_l...
8,Mechanical Production Engineer,ATA Electronics,6th of October,Giza,Egypt,[Full Time],Entry Level,0 - 5 Yrs of Exp,"[Manufacturing/Production, Engineering - Mecha...",Responsible for achieving production plan for ...,Experience 0 - to 5 years in metal forming and...,https://images.wuzzuf-data.net/files/company_l...
9,Senior Data Engineer,Worex,Maadi,Cairo,Egypt,[Full Time],Experienced,3 - 5 Yrs of Exp,"[IT/Software Development, Engineering - Teleco...","New data integrations, DataOps, troubleshootin...","Results-oriented, determined and having a self...",https://images.wuzzuf-data.net/files/company_l...


In [14]:
machine_learning_wuzzuf.experience[4]

'2 - 4 Yrs of Exp'

In [13]:
machine_learning_wuzzuf.to_csv('machine learning wuzzuf.csv',index=False,encoding='utf-8')